Web Apps with Spyre
---

Once you've collected data and done some analysis you'll probably want to present it to others. As we've already seen, Jupyter notebooks are a great way to do this, but if we wanted to make our results more interactive we could put the data in a web application.

Spyre is a python library that is used to quickly build data focussed web applications. Spyre does not come pre-installed with the Anaconda distribution of Python so checkout the [Spyre github page and README](https://github.com/adamhajari/spyre) for instruction on how to install it. The README also contains a couple of short examples that you should try yourself before continuing with this module.

Once you've successfully gotten the examples for the spyre README to work, let's again turn to the `nbs.db` sqlite database to make a new app. We'll be pulling in data from two tables and joining the results in pandas. Start by querying all rows from the `artists` and `social_data` tables. Let's work on the components of the app here and then put them into a separate script.

In [1]:
import sqlite3 as lite
import pandas as pd
con = lite.connect('nbs.db')
artists_query = "select id, name from artists"
artists_df = pd.read_sql(artists_query, con)

socail_query = "select * from social_data"
social_df = pd.read_sql(socail_query, con)
artists_df.head()

,id,name
0,500,The Scenic
1,710,Our Family Name
2,744,Khamelien
3,786,Death in the Park
4,818,enter the collector


In [2]:
social_df.head()

,artist_id,FaceBookPageLikes,TwitterFollowers,YouTubeViews,InstagramLikes,InstagramFollowers
0,500,4797.0,1395.0,NaN,NaN,NaN
1,744,NaN,224.0,44287.0,NaN,NaN
2,952,4142.0,64.0,130279.0,NaN,NaN
3,1017,NaN,NaN,2966.0,NaN,NaN
4,1086,28980.0,43405.0,402054.0,56832.0,7653.0


We can now use pandas' `merge` function to merge these two dataframes so we have the social data and the artists' names in one place. The `artist_id`s from social_df match the `id`s from artist_df so we'll merge on those two columns

In [3]:
df_merged = pd.merge(artists_df, social_df, left_on='id', right_on='artist_id')
df_merged.head()

,id,name,artist_id,FaceBookPageLikes,TwitterFollowers,YouTubeViews,InstagramLikes,InstagramFollowers
0,500,The Scenic,500,4797.0,1395.0,NaN,NaN,NaN
1,744,Khamelien,744,NaN,224.0,44287.0,NaN,NaN
2,952,Shorelines End,952,4142.0,64.0,130279.0,NaN,NaN
3,1017,Hollywood Ave,1017,NaN,NaN,2966.0,NaN,NaN
4,1086,Play for keeps,1086,28980.0,43405.0,402054.0,56832.0,7653.0


Let's put these steps in a function so that we can use them again later.

In [4]:
import sqlite3 as lite
import pandas as pd

def get_social_data():
    con = lite.connect('nbs.db')
    artists_query = "select id, name from artists"
    artists_df = pd.read_sql(artists_query, con)

    socail_query = "select * from social_data"
    social_df = pd.read_sql(socail_query, con)
    df_merged = pd.merge(artists_df, social_df, left_on='id', right_on='artist_id')
    return df_merged.drop(['id', 'artist_id'], axis=1)

Now let's create a second function that takes the merged results, sorts them by one of the featuers, and returns the top artist limited by a paramater n.

In [5]:
def top_artists(network, n=10):
    """return the top n artists from network"""
    df_social = get_social_data()  # get the data from sql
    
    # sort the result by the given network in descending order
    df_sorted = df_social.sort_values(by=network, ascending=False)
    
    # limit the results to the top n artists
    df_limited = df_sorted.head(n).copy()
    
    # convert the values to integers so they look nicer
    df_limited.loc[:, network] = df_limited.loc[:, network].astype('int')
    
    # return the results but just include the values for the network of interest
    return df_limited.loc[:, ['name', network]]

top_artists('TwitterFollowers', n=7)

,name,TwitterFollowers
1881,The Amp Jackers,97933
1049,DaveRose,95106
2043,Dez Duron,89117
649,Pauly Shore,84642
1525,Doyin aka Lyrical,80197
3571,Chantelle Lee,78375
3425,Billi. B,71055


**Exercise:**
Now try to add these two functions to a spyre app that has a drop down menu for selecting a feature type and a slider that allows you to limit your results to some values between 0 and 100.

If you get stuck check out `nbs_top_artists_app.py` in this repo. You should be able to run the app from a terminal window by running 

```
$ python nbs_top_artists_app.py
```

Paste `http://127.0.0.1:8080` into your browser url box to try the app out.